In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import lanczos

M_PI = math.pi
Mc  = pam.Mc
print 'Mc=',Mc

w_vals = pam.w_vals
Norb = pam.Norb
eta = pam.eta
ed  = pam.ed
ep  = pam.ep

if Norb==3 or Norb==7:
    tpd = pam.tpd
    tpp = pam.tpp
elif Norb==9:
    pds = pam.pds
    pdp = pam.pdp
    pps = pam.pps
    ppp = pam.ppp
    
Upp = pam.Upp
if Norb==3:
    Udd = pam.Udd
elif Norb==7 or Norb==9:
    A = pam.A
    B = pam.B
    C = pam.C

#####################################
def get_ground_state(matrix,S_val,Sz_val):  
    '''
    Obtain the ground state info, namely the lowest peak in Aw_dd's component
    in particular how much weight of various d8 channels: a1^2, b1^2, b2^2, e^2
    '''            
    # in case eigsh does not work but matrix is actually small, e.g. Mc=1 (CuO4)
    M_dense = matrix.todense()
    vals, vecs = np.linalg.eigh(M_dense)
    
    # in case eigsh works:
    Neval = 15
    #vals, vecs = sps.linalg.eigsh(matrix, k=Neval, which='SA')
    
    print 'lowest eigenvalue of H = ', vals
    
    # get state components in GS and another few excited states; note that indices is a tuple
    for k in xrange(0,Neval):
        #print 'eigenvalue = ', vals[k]
        indices = np.nonzero(abs(vecs[:,k])>0.05)

        #print "Compute the weights in GS (lowest Aw peak)"
        for i in indices[0]:
            # now the basis is after basis change, namely singlets/triplets
            # but its orbital info remains the same as original states
            state = VS.get_state(VS.lookup_tbl[i])
            s1 = state['hole1_spin']
            s2 = state['hole2_spin']
            o1 = state['hole1_orb']
            o2 = state['hole2_orb']
            x1, y1 = state['hole1_coord']
            x2, y2 = state['hole2_coord']
                
            # also obtain the total S and Sz of the state
            S12  = S_val[i]
            Sz12 = Sz_val[i]

            o12 = sorted([o1,o2])
            o12 = tuple(o12)
            
            # Sz12 - 1 because the code label up as 1 and dn as 0
            # physical value of Sz should minus 1
            #print 'state ', o12[0],x1,y1,o12[1],x2,y2, 'S=',S12,'Sz=',Sz12-1, \
            #     ", weight = ", abs(vecs[i,k])**2
    return vals, vecs

def CuO4_analytical_Norb7(E):
    print '=============================================='
    print 'Obtain analytical eigenvalues of CuO4 problem (7-orbital):'
    print 'singlets:'
    # 1A2 
    M= [[A+4.*B+2.*C,    -2.*tpd], \
        [-2.*tpd,      -2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1A2: ',vals

    # 1B2 
    fac = sqrt(3)
    M= [[A+2.*C,    2.*B*fac,     2.*tpd/fac], \
        [2.*B*fac,  A+B+2.*C,     0], \
        [2.*tpd/fac,    0,        2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1B2: ',vals

    # 1B1 
    fac = sqrt(3)
    M= [[A+2.*C,    2.*B*fac,     2.*tpd/fac,   -2.*tpd,       0,          0], \
        [2.*B*fac,  A+B+2.*C,     0,               0,          0,          0], \
        [2.*tpd/fac,    0,        2.*tpp+ep,       0,       -2.*tpd,       0], \
        [-2.*tpd,       0,        0,            -2.*tpp+ep,  2.*tpd/fac,   0], \
        [0,             0,        -2.*tpd,       2.*tpd/fac, 2.*ep,        0], \
        [0,             0,        0,               0,          0,        2.*ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1B1: ',vals
    
    # 1Ea
    fac = sqrt(3)
    M= [[A+B+2.*C,    -B*fac,     -3.*B,     -2.*tpd,     0], \
        [-B*fac,   A+3.*B+2.*C,   -B*fac,       0,      2.*tpd/fac], \
        [-3.*B,       -B*fac,    A+B+2.*C,      0,        0], \
        [-2.*tpd,       0,            0,    -2.*tpp+ep,   0], \
        [0,          2.*tpd/fac,      0,        0,      2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1E: ',vals

    # 1Eb
    fac = sqrt(3)
    M= [[A+B+2.*C,    B*fac,     3.*B,     -2.*tpd,     0], \
        [B*fac,   A+3.*B+2.*C,   -B*fac,       0,      2.*tpd/fac], \
        [3.*B,       -B*fac,    A+B+2.*C,      0,        0], \
        [-2.*tpd,       0,           0,    -2.*tpp+ep,   0], \
        [0,         2.*tpd/fac,      0,        0,      2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1E: ',vals

    # 1A1
    fac = sqrt(2)
    fac1 = sqrt(3)
    E1 = 4.*tpp+2.*ep
    E2 = -4.*tpp+2.*ep
    M= [[A+4.*B+3.*C,  4.*B+C,       4.*B+C,    fac*(B+C),      2.*fac*tpd/fac1,       0,       0,       0,       0], \
        [4.*B+C,       A+4.*B+3.*C,  C,      fac*(3.*B+C),       0,         -2.*fac*tpd,   0,       0,       0], \
        [4.*B+C,       C,            A+4.*B+3.*C,   fac*(3.*B+C),   0,       0,            0,       0,       0], \
        [fac*(B+C),   fac*(3.*B+C),  fac*(3.*B+C),  A+7.*B+4.*C,    0,       0,            0,       0,       0], \
        [2.*fac*tpd/fac1,   0,            0,          0,        2.*tpp+ep,   0,     2.*fac*tpd/fac1,      0,       0], \
        [0,         -2.*fac*tpd,     0,                0,           0,    -2.*tpp+ep,      0,    -2.*fac*tpd, 0], \
        [0,            0,            0,                0,       2.*fac*tpd/fac1,  0,            E1,      0,        0], \
        [0,            0,            0,                0,           0,    -2.*fac*tpd,      0,      E2,       0], \
        [0,            0,            0,                0,           0,       0,             0,      0,       2.*ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1A1: ',vals
    
    # other    
    fac = sqrt(3)
    M= [[ep,          2.*tpd/fac,   0,   0], \
        [2.*tpd/fac,  2.*tpp+2.*ep, 0,   0], \
        [0,   0,  -2.*tpp+2.*ep,   -2.*tpd], \
        [0,   0,    -2.*tpd,     ep]]
    vals, vecs = np.linalg.eigh(M)
    print 'other: ',vals
    
    fac = sqrt(3)
    M= [[ep,          2.*tpd/fac,   0,   0], \
        [2.*tpd/fac,  2.*tpp+2.*ep, 0,   0], \
        [0,   0,  -2.*tpp+2.*ep,   -2.*tpd], \
        [0,   0,    -2.*tpd,     ep]]
    vals, vecs = np.linalg.eigh(M)
    print 'other: ',vals
    
    print '=============================================='
    print 'triplets:'
    # 3A2 
    M= [[A+4.*B,    6.*B,  -2.*tpd], \
        [6.*B,     A-5.*B,    0], \
        [-2.*tpd,   0,   -2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3A2: ',vals
    
    # 3B1
    fac = sqrt(3)
    M= [[A-8.*B,      2.*tpd/fac,   -2.*tpd,      0], \
        [2.*tpd/fac,  2.*tpp+ep,        0,      -2.*tpd], \
        [-2.*tpd,        0,        -2.*tpp+ep,   2.*tpd/fac], \
        [0,           -2.*tpd,      2.*tpd/fac,  2.*ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3B1: ',vals
    
    # 3B2 
    fac = sqrt(3)
    M= [[A-8.*B,       2.*tpd/fac], \
        [2.*tpd/fac,   2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3B2: ',vals
    
    # 3Ea
    fac = sqrt(3)
    M= [[A-5.*B,    -3.*B*fac,      3.*B,     -2.*tpd,     0], \
        [-3.*B*fac,    A+B,   -3.*B*fac,       0,      2.*tpd/fac], \
        [3.*B,       -3.*B*fac,    A-5.*B,      0,        0], \
        [-2.*tpd,       0,            0,    -2.*tpp+ep,   0], \
        [0,          2.*tpd/fac,      0,        0,      2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3E: ',vals
    
    # 3Eb
    fac = sqrt(3)
    M= [[A-5.*B,    3.*B*fac,      -3.*B,     -2.*tpd,     0], \
        [3.*B*fac,    A+B,   -3.*B*fac,       0,      2.*tpd/fac], \
        [-3.*B,       -3.*B*fac,    A-5.*B,      0,        0], \
        [-2.*tpd,       0,            0,    -2.*tpp+ep,   0], \
        [0,          2.*tpd/fac,      0,        0,      2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3E: ',vals
    
    # 3A1
    fac = sqrt(3)
    M= [[2.*tpp+ep,    0], \
        [0,   -2.*tpp+ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3A1: ',vals
    
    # other         
    fac = sqrt(3)
    M= [[ep,          2.*tpd/fac,   0,   0], \
        [2.*tpd/fac,  2.*tpp+2.*ep, 0,   0], \
        [0,   0,  -2.*tpp+2.*ep,   -2.*tpd], \
        [0,   0,    -2.*tpd,     ep]]
    vals, vecs = np.linalg.eigh(M)
    print 'other: ',vals
    
    fac = sqrt(3)
    M= [[ep,          2.*tpd/fac,   0,   0], \
        [2.*tpd/fac,  2.*tpp+2.*ep, 0,   0], \
        [0,   0,  -2.*tpp+2.*ep,   -2.*tpd], \
        [0,   0,    -2.*tpd,     ep]]
    vals, vecs = np.linalg.eigh(M)
    print 'other: ',vals
    
def CuO4_analytical_Norb9(E):
    ps1 = ppp-pps
    ps2 = ppp+pps
    
    print '=============================================='
    print 'Obtain analytical eigenvalues of CuO4 problem (9-orbital):'
    print 'singlets:'
    
    # 1A2 
    c = sqrt(3)
    M= [[A+4.*B+2.*C,    -c*pds,     2.*pdp,    0], \
        [-c*pds,        -ps2+ep,     0,        2.*pdp], \
        [2.*pdp,           0,        ps2+ep,    -c*pds], \
        [0,               2.*pdp,        -c*pds,  2.*ep]]
    vals, vecs = np.linalg.eigh(M)
    print '1A2: ',vals

    # 1B2 
    c = sqrt(3)
    M= [[A+2.*C,    2.*c*B,     -pds,    2.*pdp,    0], \
        [2.*c*B,    A+B+2.*C,    0,        0,        0], \
        [-pds,          0,     ps2+ep,   0,    2.*pdp], \
        [2.*pdp,        0,     0,       ps2+ep,   -pds], \
        [0,             0,   2.*pdp,     -pds,  2.*ep+2.*ps2]]
    vals, vecs = np.linalg.eigh(M)
    print '1B2: ',vals
    
    print '=============================================='
    print 'triplets:'
    # 3A2 
    M= [[A+4.*B,    6.*B,     -c*pds,    2.*pdp,    0], \
        [6.*B,    A-5.*B,      0,        0,        0], \
        [-c*pds,       0,     -ps2+ep,   0,    2.*pdp], \
        [2.*pdp,        0,     0,       ps2+ep,   -c*pds], \
        [0,             0,   2.*pdp,     -c*pds,  2.*ep]]
    vals, vecs = np.linalg.eigh(M)
    print '3A2: ',vals
    
    # 3B2
    M= [[A-8.*B,    -pds,     2.*pdp,    0], \
        [-pds,      ps2+ep,     0,        2.*pdp], \
        [2.*pdp,      0,        ps2+ep,    -pds], \
        [0,        2.*pdp,     -pds,  2.*ep+2.*ps2]]
    vals, vecs = np.linalg.eigh(M)
    print '1A2: ',vals
    
if __name__ == '__main__':    
    # set up H0
    VS = vs.VariationalSpace(Mc)
    basis.count_VS(VS)
    T_pd  = ham.create_tpd_nn_matrix(VS)
    T_pp  = ham.create_tpp_nn_matrix(VS)       
    Esite = ham.create_edep_diag_matrix(VS)
    
    # Note there is no tpp prefactor that is considered in hamiltonian.py
    H0 = T_pd + T_pp + Esite
    
    # check_hermitian for H0
    #H0_max = H0.todense()
    #if ham.check_hermitian(H0_max):
    #    print 'hermitian'
        
    # set up H_int
    # for multi d-orbital cases, compute A(w) for each symmetry channel
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))
    
    d_double, p_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix(VS,d_double)
    elif pam.basis_change_type=='d_double':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix_d_double(VS,d_double)
        
    U_d = (U.conjugate()).transpose()
    H0_new = U_d.dot(H0.dot(U))
    
    #======================= start opt for Norb=3 or 7, 9 ======================================
    if Norb==3:
        Hint, Aw_state_indices = ham.create_interaction_matrix_Norb3(VS,d_double,p_double)
        H = H0_new + Hint
        H.tocsr()
        
        if pam.if_get_ground_state==1:
            S_val  = np.zeros(VS.dim, dtype=int)
            Sz_val = np.zeros(VS.dim, dtype=int)
            vals, vecs = get_ground_state(H, S_val, Sz_val)
                
        # For debug: check if H is hermitian: only practical for small Mc, namely small matrix
        #out_mat = H.todense()
        #assert(ham.check_hermitian(out_mat)==True)
        
        Nstate = len(Aw_state_indices)
        for j in range(0,Nstate):
            index = Aw_state_indices[j]
            Aw = getAw(H,index,VS,w_vals)      

            state = VS.get_state(VS.lookup_tbl[index])
            s1 = state['hole1_spin']
            s2 = state['hole2_spin']
            o1 = state['hole1_orb']
            o2 = state['hole2_orb']
            #subplot(Nstate,1,j+1)
            plt.plot(w_vals, Aw, Ms[j], linewidth=1, label=tuple([s1,o1,s2,o2]))

            maxval = max(Aw)
            #xlim([-6.5,12])
            #ylim([0,maxval])
            ylim([0,0.5])
            #ylabel('$A(\omega)$',fontsize=14)
            #text(0.45, 0.1, '(a)', fontsize=16)
            grid('on',linestyle="--", linewidth=0.5, color='black', alpha=0.5)
            legend(loc='best', fontsize=6.5, framealpha=1.0, edgecolor='black')
            if j==0:
                title(fname, fontsize=8)
            if j==Nstate-1:
                xlabel('$\omega$',fontsize=15)

        savefig("Aw_"+fname+".pdf")

        # plot Matlab data for comparison
        #a = np.loadtxt('../data/dataMatlab.txt',skiprows=1)
        #la = len(a)
        #print la
        #plt.plot(a[0:la/3-1,0], a[0:la/3-1,1], '--c', linewidth=1, label='dd, Matlab')
        #plt.plot(a[la/3:2*la/3-1,0], a[la/3:2*la/3-1,1], '--m', linewidth=1, label='dp, Matlab')
        #plt.plot(a[2*la/3:la-1,0], a[2*la/3:la-1,1], '--k', linewidth=1, label='pp, Matlab')

    else:
        print "===================================="
        print "start calculating eigenvalues of CuO4 problem"

        # enforce sym=='ALL':
        Hint, dd_state_indices = ham.create_interaction_matrix_ALL_syms(VS,d_double,p_double,S_val, Sz_val)
        H = H0_new + Hint  
        H.tocsr()
        Evals, Evecs = get_ground_state(H, S_val, Sz_val)

        if Norb==7:
            CuO4_analytical_Norb7(Evals)
        elif Norb==9:
            CuO4_analytical_Norb9(Evals)

Populating the interactive namespace from numpy and matplotlib
Cu_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px']
O2_orbs =  ['py']
symmetries =  ['1A1']
E_1S =  12.66
E_1G =  8.26
E_1D =  7.21
E_3P =  7.55
E_3F =  5.3
Mc= 1
VS.dim =  153
No. of states with count_upup, count_updn, count_dnup, count_dndn: 36 55 26 36
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_edep_diag_matrix
len(d_list) 45 Nd= 45
start calculating eigenvalues of CuO4 problem


UnboundLocalError: local variable 'AorB_sym' referenced before assignment